In [ ]:
# ## dependencies for colab
# !git clone https://__TOKEN_GIT__:@github.com/DanielSc4/RL-on-LM.git
# %cd RL-on-LM/
# !pip install -r requirements.txt

## 0. 🗽 Imports

In [2]:
import torch
from src.utils import device_selector, gen_data
from src.ToxicityMeter import ToxicityMeter

Using MPS (Apple Silicon GPU)


#### Choose model

In [3]:
# model_id = 'EleutherAI/gpt-neo-125m'
model_id = 'EleutherAI/pythia-70m'
# model_id = 'EleutherAI/pythia-2.8b'
# model_id = 'MBZUAI/LaMini-GPT-124M'
# model_id = 'MBZUAI/LaMini-GPT-774M'
# model_id = 'MBZUAI/LaMini-GPT-1.5B'

reward_model_id = 'facebook/roberta-hate-speech-dynabench-r4-target'

In [4]:
load_dtype = '8-bit'
toxicity_meter = ToxicityMeter(
    model_id = model_id,
    load_dtype = 'fp32', # can be ['8-bit', 'bf16',]
    toxicity_model_id = reward_model_id,
)

custom_prompt = ('Donald Trump once said: "{prompt}')

loader = gen_data(
    tokenizer = toxicity_meter.tokenizer, 
    max_len = 128 + len(custom_prompt),
    custom_prompt = custom_prompt,
    batch_size = 8,
)

toxicity_df = toxicity_meter.measure_toxicity(loader = loader)
toxicity_df

Using MPS (Apple Silicon GPU)


Found cached dataset json (/Users/daniel/.cache/huggingface/datasets/allenai___json/allenai--real-toxicity-prompts-eb8779dd2693db47/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/150 [00:00<?, ?it/s]/Users/daniel/miniforge3/envs/torch_new/lib/python3.9/site-packages/transformers/generation/utils.py:686: UserWarning: MPS: no support for int64 repeats mask, casting it to int32 (Triggered internally at /Users/runner/miniforge3/conda-bld/pytorch-recipe_1680607667584/work/aten/src/ATen/native/mps/operations/Repeat.mm:236.)
  input_ids = input_ids.repeat_interleave(expand_size, dim=0)
/Users/daniel/miniforge3/envs/torch_new/lib/python3.9/site-packages/transformers/generation/beam_search.py:358: UserWarning: MPS: no support for int64 min/max ops, casting it to int32 (Triggered internally at /Users/runner/miniforge3/conda-bld/pytorch-recipe_1680607667584/work/aten/src/ATen/native/mps/operations/ReduceOps.mm:1271.)
  sent_lengths_max = sent_lengths.max().item() + 1
  1%|          | 1/150 [00:20<52:01, 20.95s/it]


KeyboardInterrupt: 

In [ ]:
toxicity_df.to_csv(f'measured_toxicity_{model_id.split("/")[-1]}{load_dtype}.csv')